# Feature Engineering

In this notebook we will select and create features to feed our ML model. For now, we will focus on the consumption data.
<br>
We will use `MLForecast` to create the lag and time-related features.

In [1]:
import pandas as pd
from mlforecast import MLForecast
from mlforecast.lag_transforms import ExpandingMean, RollingMean

In [2]:
df = pd.read_csv("../data/preprocessed/consumption_train.csv", parse_dates=["datetime"])
print(df.shape)
df.head()

(760650, 9)


,county,is_business,product_type,target,is_consumption,datetime,data_block_id,row_id,prediction_unit_id
0,0,0,1,96.590,1,2021-09-01,0,1,0
1,3,0,3,39.241,1,2021-09-01,0,25,12
2,7,1,3,453.023,1,2021-09-01,0,61,30
3,8,0,1,9.787,1,2021-09-01,0,63,31
4,3,0,1,14.964,1,2021-09-01,0,23,11


In [3]:
import sys
sys.path.append("/Users/gabriel/Documents/Git/End-to-end MLOps for Time Series")
from utils import load_config
config = load_config("../config/config.yaml")

In [9]:
forecast_horizon = config["forecast_horizon"]
n_lags = config["n_lags"]
rolling_mean_window_size = config["rolling_mean_window_size"]
n_lag_transforms = n_lags

In [124]:
forecast_horizon = 5
n_lags = 6
rolling_mean_window_size = 1
n_lag_transforms = 4

In [125]:
fcst = MLForecast(
    models=[],
    freq="h",
    lags=[i + forecast_horizon for i in range(n_lags)],
    lag_transforms={
        i + forecast_horizon: [ExpandingMean(), RollingMean(window_size=rolling_mean_window_size)]
        for i in range(n_lag_transforms)
    },
    date_features=["month", "dayofweek", "hour"],
)

In [126]:
id_col="prediction_unit_id"
time_col="datetime"
target_col="target"
id_columns = [id_col, time_col, target_col]
X = fcst.preprocess(df[id_columns], id_col=id_col, time_col=time_col, target_col=target_col)

In [ ]:
X_ = X.sort_values(by=["prediction_unit_id", "datetime"])
df_ = df.sort_values(by=["prediction_unit_id", "datetime"])
df_.head()
# TODO: determine the value of `step` for which `X` and `df` are aligned according to "datetime".
# Once it's done, stack the last `n_step` values of `train` on top of `test` so that we don't lose
# the first rows of test after preprocessing.
step = forecast_horizon + max(n_lags, n_lag_transforms) - 1
X_["datetime"].head() == df_["datetime"].iloc[step:step+5]

560    True
665    True
722    True
772    True
816    True
Name: datetime, dtype: bool

In [137]:
X_["datetime"].tail() == df_["datetime"].tail()

760341    True
760405    True
760471    True
760578    True
760645    True
Name: datetime, dtype: bool

In [135]:
X_ = X.sort_values(by=["prediction_unit_id", "datetime"])
X_.head()

,prediction_unit_id,datetime,target,lag5,lag6,lag7,lag8,lag9,lag10,expanding_mean_lag5,rolling_mean_lag5_window_size1,expanding_mean_lag6,rolling_mean_lag6_window_size1,expanding_mean_lag7,rolling_mean_lag7_window_size1,expanding_mean_lag8,rolling_mean_lag8_window_size1,month,dayofweek,hour
560,0,2021-09-01 10:00:00,36.071,89.781,88.184,87.955,91.594,77.691,96.590,88.632500,89.781,88.402800,88.184,88.457500,87.955,88.625000,91.594,9,2,10
665,0,2021-09-01 11:00:00,31.147,96.481,89.781,88.184,87.955,91.594,77.691,89.753714,96.481,88.632500,89.781,88.402800,88.184,88.457500,87.955,9,2,11
722,0,2021-09-01 12:00:00,26.138,94.592,96.481,89.781,88.184,87.955,91.594,90.358500,94.592,89.753714,96.481,88.632500,89.781,88.402800,88.184,9,2,12
772,0,2021-09-01 13:00:00,37.784,77.308,94.592,96.481,89.781,88.184,87.955,88.908444,77.308,90.358500,94.592,89.753714,96.481,88.632500,89.781,9,2,13
816,0,2021-09-01 14:00:00,45.512,54.211,77.308,94.592,96.481,89.781,88.184,85.438700,54.211,88.908444,77.308,90.358500,94.592,89.753714,96.481,9,2,14


In [136]:
df_ = df.sort_values(by=["prediction_unit_id", "datetime"])
df_.head()
step = forecast_horizon + max(n_lags, n_lag_transforms) - 1
df_.iloc[step:step+5]

,county,is_business,product_type,target,is_consumption,datetime,data_block_id,row_id,prediction_unit_id
560,0,0,1,36.071,1,2021-09-01 10:00:00,0,1221,0
665,0,0,1,31.147,1,2021-09-01 11:00:00,0,1343,0
722,0,0,1,26.138,1,2021-09-01 12:00:00,0,1465,0
772,0,0,1,37.784,1,2021-09-01 13:00:00,0,1587,0
816,0,0,1,45.512,1,2021-09-01 14:00:00,0,1709,0


In [26]:
X_grouped = X.groupby(by="prediction_unit_id").tail()
X_grouped[X_grouped["prediction_unit_id"] == 0]

,prediction_unit_id,datetime,target,lag48,lag49,lag50,lag51,lag52,lag53,lag54,...,rolling_mean_lag68_window_size24,expanding_mean_lag69,rolling_mean_lag69_window_size24,expanding_mean_lag70,rolling_mean_lag70_window_size24,expanding_mean_lag71,rolling_mean_lag71_window_size24,month,dayofweek,hour
760382,0,2023-01-26 01:00:00,664.992,858.491,888.305,938.142,1036.744,1078.101,1107.538,1085.742,...,1002.854583,381.926784,998.048583,381.882409,992.468833,381.840015,989.621042,1,3,1
760403,0,2023-01-26 02:00:00,652.737,838.541,858.491,888.305,938.142,1036.744,1078.101,1107.538,...,1008.400458,381.971650,1002.854583,381.926784,998.048583,381.882409,992.468833,1,3,2
760469,0,2023-01-26 03:00:00,634.848,822.142,838.541,858.491,888.305,938.142,1036.744,1078.101,...,1016.672458,382.019796,1008.400458,381.971650,1002.854583,381.926784,998.048583,1,3,3
760565,0,2023-01-26 04:00:00,646.030,835.813,822.142,838.541,858.491,888.305,938.142,1036.744,...,1024.601667,382.075993,1016.672458,382.019796,1008.400458,381.971650,1002.854583,1,3,4
760641,0,2023-01-26 05:00:00,654.215,818.912,835.813,822.142,838.541,858.491,888.305,938.142,...,1028.034542,382.132701,1024.601667,382.075993,1016.672458,382.019796,1008.400458,1,3,5


In [ ]:
grouped = df.groupby(by="prediction_unit_id").tail()
grouped[grouped["prediction_unit_id"] == 0]

,county,is_business,product_type,target,is_consumption,datetime,data_block_id,row_id,prediction_unit_id
760382,0,0,1,664.992,1,2023-01-26 01:00:00,512,1617781,0
760403,0,0,1,652.737,1,2023-01-26 02:00:00,512,1617913,0
760469,0,0,1,634.848,1,2023-01-26 03:00:00,512,1618045,0
760565,0,0,1,646.030,1,2023-01-26 04:00:00,512,1618177,0
760641,0,0,1,654.215,1,2023-01-26 05:00:00,512,1618309,0


# Feature engineering function

In [ ]:
def feature_engineering(
    df,
    id_col="prediction_unit_id",
    time_col="datetime",
    target_col="target",
    forecast_horizon=48,
    n_lags=24,
    rolling_mean_window_size=24,
    inference=False,
):
    if inference == True:  # add rows between the last recorded value and the target_col
        unique_ids = df[id_col].unique()
        for id in unique_ids:
            county = df.loc[df["prediction_unit_id"]==id, "county"].iloc[0]
            is_business = df.loc[df["prediction_unit_id"]==id, "is_business"].iloc[0]
            product_type = df.loc[df["prediction_unit_id"]==id, "product_type"].iloc[0]
            new_rows = pd.DataFrame(
                {
                    id_col: id,
                    time_col: pd.date_range(
                        df[time_col].iloc[-1], periods=forecast_horizon, freq="h"
                    ),
                    target_col: -99,  # can't be None
                    "county": county,
                    "is_business": is_business,
                    "product_type": product_type,
                },
                index=range(df.index.stop, df.index.stop+forecast_horizon)
            )
            df = pd.concat((df, new_rows))

    fcst = MLForecast(
        models=[],
        freq="h",
        lags=[i + forecast_horizon for i in range(n_lags)],
        lag_transforms={
            i + forecast_horizon: [ExpandingMean(), RollingMean(window_size=rolling_mean_window_size)]
            for i in range(n_lag_transforms)
        },
        date_features=["month", "dayofweek", "hour"],
    )

    id_columns = [id_col, time_col, target_col]
    X = fcst.preprocess(df[id_columns], id_col=id_col, time_col=time_col, target_col=target_col)
    columns_to_drop = id_columns + ["data_block_id", "row_id", "is_consumption"]
    X = pd.concat([df[df.columns.drop(columns_to_drop)], X], axis=1, join="inner")
    if inference == True:
        return X.drop(columns=id_columns)
    else:
        X, y = X.drop(columns=id_columns), X[target_col]
        return X, y

# Testing the function for training

In [4]:
X, y = feature_engineering(df, inference=False)
print(X.shape)
print(y.shape)

(754728, 78)
(754728,)


In [5]:
X.head()

,county,is_business,product_type,lag48,lag49,lag50,lag51,lag52,lag53,lag54,...,rolling_mean_lag68_window_size24,expanding_mean_lag69,rolling_mean_lag69_window_size24,expanding_mean_lag70,rolling_mean_lag70_window_size24,expanding_mean_lag71,rolling_mean_lag71_window_size24,month,dayofweek,hour
5264,4,1,3,356.405,510.314,593.243,498.008,651.211,665.971,702.769,...,630.763833,608.930462,630.926875,620.69836,631.689917,632.698667,632.698667,9,5,22
5265,2,0,3,31.637,41.316,40.195,29.904,21.693,14.567,8.766,...,17.781792,18.205423,17.691250,17.90728,17.676583,17.693208,17.693208,9,5,22
5266,5,1,1,101.242,104.646,127.011,149.387,144.964,133.411,191.672,...,130.350083,126.109423,130.608667,128.76856,131.103667,130.774417,130.774417,9,5,22
5267,0,1,3,5070.692,5401.186,5844.822,6041.072,6060.432,6165.160,5353.664,...,4990.877875,5026.974692,5024.709500,5064.05332,5060.261542,5095.167500,5095.167500,9,5,22
5268,7,1,0,763.137,890.890,849.272,880.601,732.112,856.077,401.599,...,641.494667,633.809077,641.823208,637.72728,641.517083,641.470167,641.470167,9,5,22


In [6]:
y.head()

5264     368.268
5265      41.177
5266      57.411
5267    5039.402
5268     305.908
Name: target, dtype: float64

# Testing the function for inference

In [7]:
X = feature_engineering(df, inference=True)
print(X.shape)

(757752, 78)


In [8]:
X.head()

,county,is_business,product_type,lag48,lag49,lag50,lag51,lag52,lag53,lag54,...,rolling_mean_lag68_window_size24,expanding_mean_lag69,rolling_mean_lag69_window_size24,expanding_mean_lag70,rolling_mean_lag70_window_size24,expanding_mean_lag71,rolling_mean_lag71_window_size24,month,dayofweek,hour
5264,4,1,3,356.405,510.314,593.243,498.008,651.211,665.971,702.769,...,630.763833,608.930462,630.926875,620.69836,631.689917,632.698667,632.698667,9,5,22
5265,2,0,3,31.637,41.316,40.195,29.904,21.693,14.567,8.766,...,17.781792,18.205423,17.691250,17.90728,17.676583,17.693208,17.693208,9,5,22
5266,5,1,1,101.242,104.646,127.011,149.387,144.964,133.411,191.672,...,130.350083,126.109423,130.608667,128.76856,131.103667,130.774417,130.774417,9,5,22
5267,0,1,3,5070.692,5401.186,5844.822,6041.072,6060.432,6165.160,5353.664,...,4990.877875,5026.974692,5024.709500,5064.05332,5060.261542,5095.167500,5095.167500,9,5,22
5268,7,1,0,763.137,890.890,849.272,880.601,732.112,856.077,401.599,...,641.494667,633.809077,641.823208,637.72728,641.517083,641.470167,641.470167,9,5,22


In [9]:
X.tail()

,county,is_business,product_type,lag48,lag49,lag50,lag51,lag52,lag53,lag54,...,rolling_mean_lag68_window_size24,expanding_mean_lag69,rolling_mean_lag69_window_size24,expanding_mean_lag70,rolling_mean_lag70_window_size24,expanding_mean_lag71,rolling_mean_lag71_window_size24,month,dayofweek,hour
763669,11,1,0,532.796,531.700,396.956,385.780,390.776,364.798,509.426,...,458.726125,336.101742,459.245333,336.112464,459.808833,336.093559,460.309583,5,0,10
763670,11,1,0,537.549,532.796,531.700,396.956,385.780,390.776,364.798,...,457.775625,336.092077,458.726125,336.101742,459.245333,336.112464,459.808833,5,0,11
763671,11,1,0,435.205,537.549,532.796,531.700,396.956,385.780,390.776,...,457.337792,336.086865,457.775625,336.092077,458.726125,336.101742,459.245333,5,0,12
763672,11,1,0,239.252,435.205,537.549,532.796,531.700,396.956,385.780,...,459.819750,336.087833,457.337792,336.086865,457.775625,336.092077,458.726125,5,0,13
763673,11,1,0,259.745,239.252,435.205,537.549,532.796,531.700,396.956,...,460.084500,336.108153,459.819750,336.087833,457.337792,336.086865,457.775625,5,0,14


In [10]:
X.isna().sum().sum()

np.int64(0)

In [11]:
ts = df[df["prediction_unit_id"]==0].iloc[-24:]
ts

,county,is_business,product_type,target,is_consumption,datetime,data_block_id,row_id,prediction_unit_id
759180,0,0,1,737.878,1,2023-01-25 06:00:00,511,1615273,0
759242,0,0,1,800.512,1,2023-01-25 07:00:00,511,1615405,0
759293,0,0,1,797.651,1,2023-01-25 08:00:00,511,1615537,0
759385,0,0,1,784.166,1,2023-01-25 09:00:00,511,1615669,0
759404,0,0,1,741.292,1,2023-01-25 10:00:00,511,1615801,0
759468,0,0,1,690.670,1,2023-01-25 11:00:00,511,1615933,0
759552,0,0,1,646.510,1,2023-01-25 12:00:00,511,1616065,0
759623,0,0,1,673.457,1,2023-01-25 13:00:00,511,1616197,0
759658,0,0,1,679.117,1,2023-01-25 14:00:00,511,1616329,0
759727,0,0,1,732.858,1,2023-01-25 15:00:00,511,1616461,0


In [12]:
ts.reset_index(drop=True, inplace=True)
ts

,county,is_business,product_type,target,is_consumption,datetime,data_block_id,row_id,prediction_unit_id
0,0,0,1,737.878,1,2023-01-25 06:00:00,511,1615273,0
1,0,0,1,800.512,1,2023-01-25 07:00:00,511,1615405,0
2,0,0,1,797.651,1,2023-01-25 08:00:00,511,1615537,0
3,0,0,1,784.166,1,2023-01-25 09:00:00,511,1615669,0
4,0,0,1,741.292,1,2023-01-25 10:00:00,511,1615801,0
5,0,0,1,690.670,1,2023-01-25 11:00:00,511,1615933,0
6,0,0,1,646.510,1,2023-01-25 12:00:00,511,1616065,0
7,0,0,1,673.457,1,2023-01-25 13:00:00,511,1616197,0
8,0,0,1,679.117,1,2023-01-25 14:00:00,511,1616329,0
9,0,0,1,732.858,1,2023-01-25 15:00:00,511,1616461,0


In [13]:
X = feature_engineering(ts, inference=True)
print(X.shape)

(1, 78)


In [14]:
X.head()

,county,is_business,product_type,lag48,lag49,lag50,lag51,lag52,lag53,lag54,...,rolling_mean_lag68_window_size24,expanding_mean_lag69,rolling_mean_lag69_window_size24,expanding_mean_lag70,rolling_mean_lag70_window_size24,expanding_mean_lag71,rolling_mean_lag71_window_size24,month,dayofweek,hour
71,0,0,1,654.215,646.03,634.848,652.737,664.992,710.865,755.605,...,780.05175,778.680333,778.680333,769.195,769.195,737.878,737.878,1,5,4
